## SERIAL DILUTION

### Methods:

1. Evenly spaced serial dilution with stock concentration, lowest concentration, number of columns, and final volume as inputs. 

2. Evenly spaced serial dilution with dilution factor (ex. 1/10), number of columns, and final volume as inputs

3. Custom dilution concentrations (not evenly spaced) with stock concentration, list of desired concentrations, and final volume as inputs

<font color='blue'>Which serial dilution method are you using? (1, 2, or 3) Enter it below.  </font>

In [241]:
method_number = 1

***

### Variables: 
<br>
<font color='blue'>Enter the following variables. These are required for all methods.</font>

In [242]:
output_hso_filename = "serial_dilution_test1.hso"  # File name of generated .hso file

highest_concentration = 1/1000   # stock solution concentration (Enter as "None" or 0 if using method 2)

num_columns = 4              # Number of columns to use in dilution
desired_final_volume = 60   # Final Voume in wells after dilution is complete.

num_mixes = 5
do_blowoff = True    # yes = True, no = False
stock_volume = 700   # volume of stock solution in (uL)

dilution_mix_volume = 20  # TODO: remove these when mix volumes are calculated correctly below
stock_mix_volume = 50

<font color='blue'>Enter the variables for the method number you have chosen</font>
<br><br>
<font color='purple'>METHOD 1 VARIABLES:</font>

In [243]:
 lowest_concentration = 1/16000

<font color='purple'>METHOD 2 VARIABLES:</font>

In [244]:
dilution_factor = 1/10   # desired dilution factor/ratio (ex. 2-fold dilution = 1/2 = 0.5)

<font color='purple'>METHOD 3 VARIABLES:</font>

In [245]:
# enter a list of concentrations in uL (ex. [(1/2000), (1/4000), (1/16000)])
dilution_concentrations_list = [(1/2000), (1/4000), (1/40000), (1/160000)]  


***

### Code:
<br> 
<font color='purple'>SETUP AND IMPORT STATEMENTS</font>

In [246]:
import os
import sys

sys.path.append(
    os.path.abspath(
        os.path.join(os.path.abspath(os.path.dirname("__file__")), "../../src")
    )
)

import SoloSoft

from Plates import (   
    GenericPlate96Well,
    NinetySixDeepWell,
    ZAgilentReservoir_1row,
) 

<font color='purple'>CALCULATIONS</font>
<br><br>
<font color='blue'>Calculations will be printed below this cell. Check that they match your expectations</font>

In [247]:
# Variables for deciding which tip types are necessary


#* Method 1
if method_number == 1:
    dilution_factor = (lowest_concentration/highest_concentration) ** (1/num_columns)
    serial_transfer_volume = int(dilution_factor * desired_final_volume)  
    diluent_transfer_volume = int(desired_final_volume - serial_transfer_volume)
    
    # calculate final volume in each well (TODO -> put calculation into a method?)
    final_volumes = [diluent_transfer_volume]*num_columns
    final_volumes[0] += serial_transfer_volume
    for i in range(num_columns-1):
        final_volumes[i] -= serial_transfer_volume
        final_volumes[i+1] += serial_transfer_volume
        
    # decide if 50 uL tips are necessary
    diluent_50uL = False  
    serial_50uL = False
    if serial_transfer_volume < 20:
        serial_50uL = True
    if diluent_transfer_volume <20:
        diluent_50uL = True
    
    # See calculation output below
    print("Method 1:" + "\n dilution factor = " + str(dilution_factor) 
          + "\n diluent transfer volume (uL) = " + str(diluent_transfer_volume) 
          + "\n serial transfer volume (uL) = " + str(serial_transfer_volume) 
          + "\n final volume in each well (uL) = " + str(final_volumes))
          
#* Method 2
elif method_number == 2:
    serial_transfer_volume = int(dilution_factor * desired_final_volume)  
    diluent_transfer_volume = int(desired_final_volume - serial_transfer_volume)
    
    # calcualte final volume in each well (same as in method 1)
    final_volumes = [diluent_transfer_volume]*num_columns
    final_volumes[0] += serial_transfer_volume
    for i in range(num_columns-1):
        final_volumes[i] -= serial_transfer_volume
        final_volumes[i+1] += serial_transfer_volume
        
     # decide if 50 uL tips are necessary (same as in method 1)
    diluent_50uL = False  
    serial_50uL = False
    if serial_transfer_volume < 20:
        serial_50uL = True
    if diluent_transfer_volume <20:
        diluent_50uL = True
    
    # See calculation output below
    print("Method 2: \n diluent transfer volume (uL) = " + str(diluent_transfer_volume) 
          + "\n serial transfer volume (uL) = " + str(serial_transfer_volume)
          + "\n final volume in each well (uL) = " + str(final_volumes))
    
#* Method 3 
elif method_number == 3:
    num_columns = len(dilution_concentrations_list)
    serial_transfer_volumes = [0]*num_columns
    diluent_transfer_volumes = [0]*num_columns
    for i in range(len(dilution_concentrations_list)):
        if i == 0: 
            current_dilution_factor = (dilution_concentrations_list[i]/highest_concentration) 
        else:
            current_dilution_factor = (dilution_concentrations_list[i]/dilution_concentrations_list[i-1])
        serial_transfer_volumes[i] = int(current_dilution_factor * desired_final_volume)
        diluent_transfer_volumes[i]  = int(desired_final_volume - serial_transfer_volumes[i])
        
    # decide if 50 uL tips are necessary on plate deck (and save unique tip types necessary for each transfer)
    diluent_50uL = False  
    serial_50uL = False
    
    serial_tip_locations = [[200, "Position1"]]*num_columns  # 200uL tips is the default
    diluent_tip_locations = [[200, "Position1"]]*num_columns
    
    if max(serial_transfer_volumes) <= 50 and max(diluent_transfer_volumes) <= 50:
        # no 200uL tips needed - 50uL tips will be in default position1
        serial_tip_locations = [[50, "Position1"]]*num_columns  
        diluent_tip_locations = [[50, "Position1"]]*num_columns
        diluent_50uL = True
        serial_50uL = True
    else:
        for i in range(len(serial_transfer_volumes)):
            if serial_transfer_volumes[i] < 20: 
                serial_tip_locations[i] = [50, "Position7"]
                serial_50uL = True
            if diluent_transfer_volumes[i] < 20:
                diluent_tip_locations[i] = [50, "Position7"]
                diluent_50uL = True
 
    # calculate final volume in each well
    final_volumes = diluent_transfer_volumes.copy()
    for i in range(num_columns):
        final_volumes[i] += serial_transfer_volumes[i]
        if i < num_columns-1:
            final_volumes[i] -= serial_transfer_volumes[i+1]
          
    # See calculation output below
    print("Method 3: \n desired concentrations list (uL/uL) = " + str(dilution_concentrations_list) 
          + "\n diluent transfer volumes (uL) = " + str(diluent_transfer_volumes) 
          + "\n serial transfer volumes (uL) = " + str(serial_transfer_volumes)
          + "\n final volume in each well (uL) = " + str(final_volumes))
    
    
print(serial_tip_locations)
print(diluent_tip_locations)

Method 1:
 dilution factor = 0.5
 diluent transfer volume (uL) = 30
 serial transfer volume (uL) = 30
 final volume in each well (uL) = [30, 30, 30, 60]
[[200, 'Position1'], [200, 'Position1'], [50, 'Position7'], [50, 'Position7']]
[[200, 'Position1'], [200, 'Position1'], [200, 'Position1'], [200, 'Position1']]


<font color='purple'>GENERATE DECK LAYOUT (and determine what tips are necessary)</font>

In [248]:
plate_list = [
        "TipBox-Corning 200uL",
        "Empty",
        "Corning 3383",
        "Corning 3383",
        "12 Channel Reservoir",
        "Empty",
        "Empty",
        "Empty",
    ]

user_readable_plate_list = [
        "TipBox-200uL",
        "Empty - HEAT NEST",
        "Stock plate - Corning 3383",
        "Serial dilution plate - Corning 3383",
        "Diluent plate - 12 Channel Reservoir",
        "Empty",
        "Empty",
        "Empty",
    ]

tips_name_50uL = "TipBox-50uL EV-50-R-S"
tips_location_50uL = None

# decide if 50 uL tips are necessary 
if (diluent_50uL == False and serial_50uL == True) or (diluent_50uL == True and serial_50uL == False):
    plate_list[6] = tips_name_50uL
    user_readable_plate_list[6] = "TipBox-50uL"
    tips_location_50uL = "Position7"
elif diluent_50uL == True and serial_50uL == True:
    plate_list[0] = tips_name_50uL
    user_readable_plate_list[0] = "TipBox-50uL"
    tips_location_50uL = "Position1"
    
    



***

<font color='blue'>----- **PLEASE SET UP THE DECK ACCORDINGLY** ------</font>

In [249]:
print("DECK LAYOUT:")
for i in range(len(user_readable_plate_list)):
    print(str(i+1) + " -> " + str(user_readable_plate_list[i]))
    
# TODO: add in some notes to the user about defalt positions and column numbers for each plate (in beginning?)

DECK LAYOUT:
1 -> TipBox-200uL
2 -> Empty - HEAT NEST
3 -> Stock plate - Corning 3383
4 -> Serial dilution plate - Corning 3383
5 -> Diluent plate - 12 Channel Reservoir
6 -> Empty
7 -> Empty
8 -> Empty


***

<font color='purple'>CALCULATE MIXING VOLUMES (AND BLOWOFF VOLUME?)</font>

In [250]:
# TODO: refactor this to prevent repeated code
# TODO: account for volume taken from stock solution well each time !!!!!

stock_mix_volumes = [0]*num_columns   
dilution_mix_volumes = [0]*num_columns 


# Methods 1 and 2
if not method_number == 3:
    if tips_location_50uL == None:
        stock_mix_volumes = [int(.6 * 200)]*num_columns if stock_volume > 200 else [int(.6 * stock_volume)]*num_columns
        for i in range(num_columns):
            dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 200 else int(200*.6)
    
    elif diluent_50uL == True and serial_50uL == False:
        if not method_number == 3: 
            stock_mix_volumes = [int(.6*50)]*num_columns if stock_volume > 50 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 200 else int(200*.6)
    
    elif diluent_50uL == False and serial_50uL == True:
        if not method_number == 3: 
            stock_mix_volumes = [int(.6*200)]*num_columns if stock_volume > 200 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 50 else int(50*.6)

    elif diluent_50uL == True and serial_50uL == True:
        if not method_number == 3:  
            stock_mix_volumes = [int(.6*50)]*num_columns if stock_volume > 50 else [int(.6*stock_volume)]*num_columns
            for i in range(num_columns):
                dilution_mix_volumes[i] = int(final_volumes[i]*.6) if (final_volumes[i]*.6) < 50 else int(50*.6)

# Method 3
elif method_number == 3:
    for i in range(num_columns):
        # stock mix volumes
        if stock_volume > diluent_tip_locations[i][0]:
            stock_mix_volumes[i] = int(diluent_tip_locations[i][0] * .6)
        else: 
            stock_mix_volumes[i] = int(stock_volume * .6)  # TODO: make this calculate the current stock volume!!!
        #dilution mix volumes
        if final_volumes[i] <= serial_tip_locations[i][0]:
            dilution_mix_volumes[i] = int(final_volumes[i]*.6)
        else:
            dilution_mix_volumes[i] = int(.6*serial_tip_locations[i][0])   

Stock volume:
700
--------
Stock mixing volumes:
[120, 120, 120, 120]
Dilution mixing volumes:
[18, 18, 18, 36]
-----------
Final Volumes
[30, 30, 30, 60]
Dilutent Tip Locations:
[[200, 'Position1'], [200, 'Position1'], [200, 'Position1'], [200, 'Position1']]
Serial Tip Locations:
[[200, 'Position1'], [200, 'Position1'], [50, 'Position7'], [50, 'Position7']]


<font color='purple'>GENERATE SOLOSOFT .HSO FILE</font>

In [254]:
# initialize soloSolft
soloSoft = SoloSoft.SoloSoft(
    filename=output_hso_filename,
    plateList=plate_list,
)

# METHODS 1 and 2
if method_number == 1 or method_number == 2:
    
    # determine the correct tip box locations
    diluent_tip_location = "Position1" if diluent_50uL == False else tip_location_50uL
    serial_tip_location = "Position1" if serial_50uL == False else tip_location_50uL
    
    # distribute diluent into all required wells 
    soloSoft.getTip(diluent_tip_location) 
    for i in range(1,num_columns+1):  # maybe add blowoff
        soloSoft.aspirate(  
            position="Position5", 
            aspirate_volumes=ZAgilentReservoir_1row().setColumn(1, diluent_transfer_volume),
            aspirate_shift = [0,0,4] # larger z-shift needed for 12 channel reservoir
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i+1, diluent_transfer_volume), 
            dispense_shift=[0,0,2], 
        )
    
    # get the correct size tips for the serial dilution steps
    if not serial_tip_location == diluent_tip_location:
        soloSoft.getTips(serial_tip_location)
        
    # dilute into first column from stock solution
    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volume),  # TODO make sure the user places stock solution in this location
        aspirate_shift = [0,0,2], 
        mix_at_start=True,
        mix_cycles=num_mixes,
        mix_volume=stock_mix_volumes[0],
        dispense_height=2,
    )
    soloSoft.dispense(
        position="Position4",
        dispense_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volume), 
        dispense_shift=[0,0,2], 
        mix_at_finish=True, 
        mix_cycles=num_mixes, 
        mix_volume=dilution_mix_volumes[0],
        aspirate_height=2,
    )

    # serial dilute into the remaining columns
    for i in range(1,num_columns):  
        soloSoft.aspirate(
            position="Position4", 
            aspirate_volumes=GenericPlate96Well().setColumn(i, serial_transfer_volume),  # TODO make sure the user places stock solution in this location
            aspirate_shift = [0,0,2], 
            mix_at_start=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volumes[i-1],
            dispense_height=2,
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i+1, serial_transfer_volume), 
            dispense_shift=[0,0,2], 
            mix_at_finish=True, 
            mix_cycles=num_mixes, 
            mix_volume=dilution_mix_volumes[i],
            aspirate_height=2,
        )

    soloSoft.shuckTip()
    soloSoft.savePipeline()

# METHOD 3
elif method_number == 3: 
    
    print(diluent_tip_locations)

    # dispense predetermined differing amounts of diluent to each well
    soloSoft.getTip(diluent_tip_locations[0][1])  # the first diluent tip locaiton 
    for i in range(1,num_columns+1): # +1 means 1,2,...,num_columns
        soloSoft.aspirate(
            position="Position5", 
            aspirate_volumes=ZAgilentReservoir_1row().setColumn(1, diluent_transfer_volumes[i-1]),
            aspirate_shift=[0,0,4],
        )
        soloSoft.dispense(
            position="Position3", 
            dispense_volumes=GenericPlate96Well().setColumn(i, diluent_transfer_volumes[i-1]), 
            dispense_shift=[0,0,2],
        )
        if not i > (num_columns-1):   # Change tip sizes if necessary
            if not diluent_tip_locations[i-1][1] == diluent_tip_locations[i][1]:
                soloSoft.getTip(diluent_tip_locations[i][1])
        
    # dispense the predetermined correct amount of stock solution into the first column
    if not diluent_tip_locations[-1][1] == serial_tip_locations[0][1]:  # switch to correct tip type of necessary
        soloSoft.getTips(serial_tip_locations[0][1])
    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volumes[0]),  # TODO make sure the user places stock solution in this location
        aspirate_shift = [0,0,2], 
        mix_at_start=True,
        mix_cycles=num_mixes,
        mix_volume=stock_mix_volumes[0],
        dispense_height=2,
    )
    soloSoft.dispense(
        position="Position4",
        dispense_volumes=GenericPlate96Well().setColumn(1, serial_transfer_volumes[0]), 
        dispense_shift=[0,0,2], 
        mix_at_finish=True, 
        mix_cycles=num_mixes, 
        mix_volume=dilution_mix_volume[0],
        aspirate_height=2,
    )

    # serial dilute into remaining columns 
    for i in range(1,num_columns): 
        if not i > (num_columns-1):   # Change tip sizes if necessary
            if not serial_tip_locations[i-1][1] == serial_tip_locations[i][1]:
                soloSoft.getTip(diluent_tip_locations[i][1])
        soloSoft.aspirate(
            position="Position4", 
            aspirate_volumes=GenericPlate96Well().setColumn(i, serial_transfer_volumes[i-1]),  # TODO make sure the user places stock solution in this location
            aspirate_shift = [0,0,2], 
            mix_at_start=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volumes[i-1],
            dispense_height=2,
        )
        soloSoft.dispense(
            position="Position4",
            dispense_volumes=GenericPlate96Well().setColumn(i+1, serial_transfer_volumes[i-1]), 
            dispense_shift=[0,0,2], 
            mix_at_finish=True, 
            mix_cycles=num_mixes, 
            mix_volume=dilution_mix_volume[i],
            aspirate_height=2,
        )

    soloSoft.shuckTip()
    soloSoft.savePipeline()

In [ ]:
# Run protocol in softlinx and auto start/remote start the program